In [ ]:
# This file is to extract emotion from data 

from transformers import pipeline
classifier = pipeline("sentiment-analysis", model="michellejieli/emotion_text_classifier")



[{'label': 'neutral', 'score': 0.8817856907844543}]

In [26]:
emotion_to_index = {
    'angry':0,
    'anger':0,
    'disgust':1,
    'fear':2,
    'fearful':2,
    'happy':3,
    'joy':3,
    'sad':4,
    'sadness':4,
    'surprise':5,
    'neutral':6,
    'calm':6,
    'surprised':5
}

In [24]:
import json

data = json.load(open('sarcasm_data.json'))


In [28]:
result = {}
for key in data:
    result[key] = {}
    result[key]['text'] = [0,0,0,0,0,0,0]
    emotion = classifier(data[key]['utterance'])[0]['label']
    result[key]['text'][emotion_to_index[emotion]] += 1

In [2]:
# Load model directly
from transformers import AutoImageProcessor, AutoModelForImageClassification

image_processor = AutoImageProcessor.from_pretrained("dima806/facial_emotions_image_detection")
image_model = AutoModelForImageClassification.from_pretrained("dima806/facial_emotions_image_detection")

preprocessor_config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

c:\Users\yingq\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\yingq\.cache\huggingface\hub\models--dima806--facial_emotions_image_detection. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config.json:   0%|          | 0.00/907 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/343M [00:00<?, ?B/s]

In [31]:
from PIL import Image
import torch
import os
from tqdm import tqdm

# Ensure CUDA is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move the model to the GPU
image_model = image_model.to(device)

# Define batch size
BATCH_SIZE = 16  # Adjust based on your GPU memory

for key in tqdm(data):
    images = os.listdir(f"video/Frames/{key}")
    emotion = [0, 0, 0, 0, 0, 0, 0]
    
    # Process images in batches
    for i in range(0, len(images), BATCH_SIZE):
        batch_images = images[i:i + BATCH_SIZE]
        batch_inputs = []
        
        # Preprocess all images in the batch
        for image in batch_images:
            img = image_processor(
                Image.open(f"video/Frames/{key}/{image}").convert("RGB"),
                return_tensors="pt"
            )['pixel_values']  # Extract pixel values
            batch_inputs.append(img)
        
        # Stack images into a single tensor batch
        batch_inputs = torch.cat(batch_inputs, dim=0).to(device)
        
        # Run inference on GPU
        with torch.no_grad():
            predictions = image_model(pixel_values=batch_inputs)
        
        # Process predictions
        predicted_class_indices = predictions.logits.argmax(-1).tolist()
        labels = image_model.config.id2label  # Get label mapping from model config
        
        for idx in predicted_class_indices:
            predicted_label = labels[idx]
            emotion[emotion_to_index[predicted_label]] += 1
    
    # Determine the dominant emotion
    max_index = emotion.index(max(emotion))
    result[key]['image'] = [0, 0, 0, 0, 0, 0, 0]
    result[key]['image'][max_index] = 1


100%|██████████| 690/690 [24:41<00:00,  2.15s/it]


In [5]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("audio-classification", model="ehcalabres/wav2vec2-lg-xlsr-en-speech-emotion-recognition")

model.safetensors:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

Some weights of the model checkpoint at ehcalabres/wav2vec2-lg-xlsr-en-speech-emotion-recognition were not used when initializing Wav2Vec2ForSequenceClassification: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.output.bias', 'classifier.output.weight', 'wav2vec2.encoder.pos_conv_embed.conv.weight_g', 'wav2vec2.encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at ehcalabres/wav2vec2-lg-xlsr-e

In [ ]:
for key in data:
    audio = f"audio/final/{key}.wav"
    emotion = pipe(audio)[0]['label']
    result[key]['audio'] = [0,0,0,0,0,0,0]
    result[key]['audio'][emotion_to_index[emotion]] += 1

In [33]:
json.dump(result, open('emotion.json', 'w'), indent=4)